In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
training_data = datasets.MNIST(
                    root="data",
                    train=True,
                    download=True,
                    transform=ToTensor()
                )

testing_data = datasets.MNIST(
                    root="data",
                    train=False,
                    download=True,
                    transform=ToTensor()
                )

In [ ]:
print("Number of training examples: " + str(len(training_data)))
print("Number of test instances: " + str(len(testing_data)))

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [ ]:
# Defining the model
import torch.nn as nn


class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.linear_relu_softmax_stack = nn.Sequential(
        nn.Linear(28*28*1, 350),
        nn.ReLU(),
        nn.Linear(350, 200),
        nn.ReLU(),
#         nn.Linear(200, 10),
#         nn.ReLU(),    
        nn.Softmax(dim=1),
        )
        
    def forward(self, x):
        out = self.linear_relu_softmax_stack(x)

        return out

In [ ]:
# Creating the model

model = Network()
print(model)
sample = torch.randn(1,784)
model(sample)

In [ ]:
img = img.reshape((1, 784))
img.shape
label.shape

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_data, batch_size=100, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=100, shuffle=False)
for i,(img,label) in enumerate(train_dataloader):
    # Flatten the array
    img = img.reshape((-1,784))
    print("i = {}, img = {}, label = {}".format(i,img.shape,label.shape))

In [285]:
# Hyper parameters
num_of_epochs = 20
learning_rate = 0.1
batch_size = 100

from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=batch_size, shuffle=False)

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.linear_relu_softmax_stack = nn.Sequential(
        nn.Linear(28*28, 200),
        nn.ReLU(),
        nn.Linear(200, 50),  
        nn.ReLU(),
        nn.Linear(50, 10),  
        nn.ReLU(),
            
#         Upto 86% Accuracy with 1 hidden layer, 450 nodes, 200 batch size, learning rate = 0.2
#         nn.Linear(28*28, 450),
#         nn.ReLU(),
#         nn.Linear(450, 10),  
#         nn.ReLU(),
        nn.Softmax(dim=1),
        )
        
    def forward(self, x):
        out = self.linear_relu_softmax_stack(x)

        return out
    
# Creating the model
model = Network()
# print(model)
# sample = torch.randn(1,784)
# model(sample)


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
size = len(training_data)
num_iters = len(training_data) / batch_size

for epoch in range(num_of_epochs):
    print("\n---------------------------Epoch {}:---------------------------".format(epoch+1))
    for i,(imgs,labels) in enumerate(train_dataloader):
        # Flatten the array
        imgs = imgs.reshape((-1,784))

        # Forward propagation (predictions + loss computation)
        preds = model(imgs)
        loss = loss_fn(preds, labels)

        # Backpropagation (calculating gradients + updating weights)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        #print("Epoch:{}, iteration:{}".format(epoch,i))
        
        # Print statistics
        if (i + 1) % 100 == 0:
            current_batch = i + 1
            l = loss.item()

            print("Loss : {}, \tIteration : [{}/{}]".format(l, current_batch,int(num_iters)))
print("\nTraining complete!")
print("Testing...")
with torch.no_grad():
    size = len(test_dataloader.dataset)
    correct_labels = 0
    for imgs,labels in test_dataloader:
        imgs = imgs.reshape((-1,784))
        
        # Make predictions
        preds = model(imgs)
        
        # Get the predicted labels (indexes with highest probabilities)
        _,indexes = torch.max(preds,1)
        
        # Count the number of correct labels
        correct_labels +=  (indexes == labels).type(torch.float).sum().item()
    #    print("Prediction = {}, Label = {}".format(pred.shape,label.shape))
print("Test Accuracy = {}% on {} samples".format(100 * (correct_labels / size), size))


---------------------------Epoch 1:---------------------------
Loss : 2.3018312454223633, 	Iteration : [100/600]
Loss : 2.2955424785614014, 	Iteration : [200/600]
Loss : 2.280592679977417, 	Iteration : [300/600]
Loss : 2.195711612701416, 	Iteration : [400/600]
Loss : 2.21809983253479, 	Iteration : [500/600]
Loss : 2.0511932373046875, 	Iteration : [600/600]

---------------------------Epoch 2:---------------------------
Loss : 2.028687000274658, 	Iteration : [100/600]
Loss : 1.8848451375961304, 	Iteration : [200/600]
Loss : 1.8231468200683594, 	Iteration : [300/600]
Loss : 1.7065027952194214, 	Iteration : [400/600]
Loss : 1.6983158588409424, 	Iteration : [500/600]
Loss : 1.7179397344589233, 	Iteration : [600/600]

---------------------------Epoch 3:---------------------------
Loss : 1.650485634803772, 	Iteration : [100/600]
Loss : 1.6709375381469727, 	Iteration : [200/600]
Loss : 1.7145496606826782, 	Iteration : [300/600]
Loss : 1.6970325708389282, 	Iteration : [400/600]
Loss : 1.69468